# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [90]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.model import Model
from azureml.widgets import RunDetails
from sklearn.metrics import classification_report

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import joblib


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstone-project-exp3'

experiment=Experiment(ws, experiment_name)

dataset = Dataset.get_by_name(ws, name='Mushroom Dataset')

In [49]:
cluster_name = "aml-compute"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [77]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling (
    {
        'C': choice(100, 10, 1.0, 0.1, 0.01),
        "max_iter": choice(10, 20, 50, 100, 200)
    }   
)

myenv = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      arguments=['--input-data', dataset.id],
                      compute_target=cpu_cluster,
                      environment=myenv)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=100,
                             max_concurrent_runs=4)

In [78]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

In [79]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_3332290e-321d-44ec-8f50-3241be07b953
Web View: https://ml.azure.com/runs/HD_3332290e-321d-44ec-8f50-3241be07b953?wsid=/subscriptions/baa67dbf-45d0-4d84-b662-527186361068/resourcegroups/tsetra-t332-20200511/workspaces/setra_analytics&tid=fd799da1-bfc1-4234-a91c-72b3a1cb9e26

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-03-28T21:06:42.873460][API][INFO]Experiment created<END>\n""<START>[2022-03-28T21:06:43.891881][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-03-28T21:06:44.746496][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2022-03-28T21:07:13.528801][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-03-28T21:07:13.833891][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2022-03-28T21:07:43.5248990Z][SCHEDULER][INFO]Scheduling job, id='HD_333229

{'runId': 'HD_3332290e-321d-44ec-8f50-3241be07b953',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2022-03-28T21:06:42.570726Z',
 'endTimeUtc': '2022-03-28T21:26:15.200329Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4034bf8a-e9ec-42dc-b867-21af7ac58a79',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1068-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '25',
  'score': '0.8250859669232029',
  'best_child_run_id': 'HD_3332290e-321d-44ec-8f50-3241be07b953_8',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://t332teststrorage.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_3332290e-321d-44ec-8f50-3241be07b953/azure

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [80]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [81]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:', best_run_metrics['Regularization Strength:'])
print('\n max_iter:', best_run_metrics['Max iterations:'])

Best Run Id:  HD_3332290e-321d-44ec-8f50-3241be07b953_8

 Accuracy: 0.8250859669232029

 C: 10.0

 max_iter: 200


In [85]:
best_run.download_file('outputs/hyperdrive_model.joblib', 'models/hp_best_model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [88]:
model = best_run.register_model(model_name='hyperdrive-model-mushroom', model_path='outputs/hyperdrive_model.joblib')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
model.delete()
cpu_cluster.delete()


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

